# 5. Analysis

Time for some machine learning!

In [1]:
# set up some dependencies
import json, time, random, csv
import geopandas as gp
import pandas as pd
import matplotlib.pyplot as plt
import descartes # for plotting with geopanadas

In [2]:
# set up path to app credentials - see exploration/README.md
%env GOOGLE_APPLICATION_CREDENTIALS=../google_app_credentials.json

# set up bigquery client
from google.cloud import bigquery
bq = bigquery.Client()

env: GOOGLE_APPLICATION_CREDENTIALS=../google_app_credentials.json


In [3]:
# grab valid years and check BigQuery connection
resp = bq.query('''
    SELECT DISTINCT do_date AS year
    FROM `eosc410-project.data.acs_cbsa_20*`
    ORDER BY do_date ASC
''')
years = [row["year"] for row in resp]

In [7]:
'''
Gets the GeoID from the combined ID
'''
def combined_id_to_geoid(combined_id: str) -> str:
    parts = combined_id.split('.')
    if len(parts) < 1:
        return ''
    return parts[0]

In [8]:
# load up aggregated data - refer to _data/acs/README.md and _data/epa/README.md
acs_ts_flat = pd.read_csv('../_data/acs/timeseries_flat.csv')
acs_ts_vert = pd.read_csv('../_data/acs/timeseries_vert.csv')
epa_ts_flat = pd.read_csv('../_data/epa/timeseries_flat.csv')
epa_ts_vert = pd.read_csv('../_data/epa/timeseries_vert.csv')

# take a peak at what's up
print('==== ACS Aggregates\n')
print(acs_ts_flat.dtypes, '\n')
print(acs_ts_vert.dtypes, '\n')
print('==== EPA Aggregates\n')
print(epa_ts_flat.dtypes, '\n')
print(epa_ts_vert.dtypes, '\n')

==== ACS Aggregates

Unnamed: 0                                        int64
10140.epa.commute_10_29_mins                    float64
10140.epa.commute_30_59_mins                    float64
10140.epa.commute_over_60_mins                  float64
10140.epa.commute_under_10_mins                 float64
                                                 ...   
43220.epa.rent_over_50_percent                  float64
43220.epa.rent_under_25_percent                 float64
43220.epa.some_college_and_associates_degree    float64
43220.epa.total_pop                             float64
43220.epa.walked_to_work                        float64
Length: 15598, dtype: object 

Unnamed: 0       int64
combined_id     object
2007           float64
2008           float64
2009           float64
2010           float64
2011           float64
2012           float64
2013           float64
2014           float64
2015           float64
2016           float64
2017           float64
2018           float64
dtype: obj

In [ ]:
# TODO